In [1]:
import requests

# Send a GET request to the desired API URL
response = requests.get('https://api.hearthstonejson.com/v1/190920/frFR/cards.collectible.json')

# Parse the response and print it
data = response.json()
#print(data)


In [18]:
def findLegendaire(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'LEGENDARY':
            compteur+= 1
    return compteur

def findEpic(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'EPIC':
            compteur+= 1
    return compteur

def findRare(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'RARE':      
            compteur+= 1
    return compteur


def findCommon(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'COMMON':
            compteur+= 1
    return compteur

def findFree(data) :
    compteur =0
    for card in data :
        if card["rarity"] == 'FREE':
            compteur+= 1
    return compteur



def dataSetInfo(data):
    long = len(data)
    print("il y a ",long," cartes au total")
    print(findLegendaire(data)," légendaire sois ",(findLegendaire(data)/long)*100, "% du dataset")
    print(findEpic(data)," epic sois ",(findEpic(data)/long)*100, "% du dataset")
    print(findRare(data)," rare sois ",(findRare(data)/long)*100, "% du dataset")
    print(findCommon(data)," commune sois ",(findCommon(data)/long)*100, "% du dataset")
    print(findFree(data)," free sois ",(findFree(data)/long)*100, "% du dataset")

def find(data, attibut) :
    compteur = {}
    for card in data :
        valeur=card.get(attibut)
        if valeur in compteur:
            compteur[valeur]+=1
        else:
            compteur[valeur]=1
    return( compteur)
    

dataSetInfo(data)

il y a  5845  cartes au total
855  légendaire sois  14.627887082976903 % du dataset
887  epic sois  15.175363558597091 % du dataset
1446  rare sois  24.739093242087254 % du dataset
2029  commune sois  34.713430282292556 % du dataset
628  free sois  10.744225834046194 % du dataset


In [14]:
import csv

def save_csv(dictio, attributs, name):
    with open(name, mode='w', newline='', encoding='utf-8') as fichier_csv:
        writer = csv.DictWriter(fichier_csv, fieldnames=attributs)
        writer.writeheader()
        for objet in dictio:
            ligne = {attr: objet.get(attr, '') for attr in attributs}
            writer.writerow(ligne)


save_csv(data,["id","name","artist","cardClass","cost","attack","health","set","type","rarity"], "dataMK1.csv")
save_csv(data,["faction"], "data_test_trash.csv")
# retiré faction car 5600 et quelque variable était NaN
# retiré collectible, que des true

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np

In [15]:
hs = pd.read_csv("dataMK1.csv")
hs.head()

,id,name,artist,cardClass,cost,attack,health,set,type,rarity
0,AT_001,Lance de flammes,Nutthapon Petchthai,MAGE,5.0,NaN,NaN,TGT,SPELL,COMMON
1,AT_002,Effigie,Tooth,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE
2,AT_003,Héros défunt,Arthur Bozonnet,MAGE,2.0,3.0,2.0,TGT,MINION,RARE
3,AT_004,Déflagration des Arcanes,Gabor Szikszai,MAGE,1.0,NaN,NaN,TGT,SPELL,EPIC
4,AT_005,Métamorphose : sanglier,Mike Sass,MAGE,3.0,NaN,NaN,TGT,SPELL,RARE


In [12]:
val=["attack","health"]

def comptNaN(fichier, col):
    colselect=hs[col]
    compteur=colselect.isna().sum()
    return compteur

comptNaN(hs, val)

attack    2094
health    1831
dtype: int64